In [6]:
from transformers import pipeline, set_seed
from transformers import TextStreamer, pipeline
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig)
from peft import LoraConfig, get_peft_model, PeftModel
import torch

mdataset_path = "/root/JurisGPT/rawdata/laboral/sumariosbigdb/sumariosdb.json" # in JSON
model_path = "meta-llama/Llama-2-13b-chat-hf"
tokenizer_path = "meta-llama/Llama-2-13b-chat-hf"
trained_model_checkpoints_save_path = "/root/checkpoints/"
trained_model_save_path = "/root/LLM-models/Llama-2-13b-hf-juris"
trained_model_save_path_final  = "/root/LLM-models/Llama-2-13b-hf-juris/"
#model_path = "meta-llama/Llama-2-13b-hf"
#tokenizer_path = "meta-llama/Llama-2-13b-hf"

#model_path = "HuggingFaceH4/zephyr-7b-beta"
#tokenizer_path = "HuggingFaceH4/zephyr-7b-beta"



#model_path = "/home/harpo/CEPH/LLM-models/Llama-2-7b-chat-hf/"
#tokenizer_path = "/home/harpo/CEPH/LLM-models/Llama-2-7b-chat-hf/"

quantize = True
bnb_config = BitsAndBytesConfig(
#load_in_8bit=True
load_in_4bit=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype= getattr(torch, "float16"),
bnb_4bit_use_double_quant=True,
)

quantization_config = bnb_config if quantize else None

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=bnb_config,
        #device_map={"": 0}, # for setting up a particular GPU
        device_map="auto", # for multiple GPUs
        trust_remote_code=True
)
#merged_model = model.merge_and_unload() #Not supporting 8bit models
model = PeftModel.from_pretrained(model,f'{trained_model_save_path}/lora',local_files_only=True)
print(f"[] Saving Model at {trained_model_save_path_final}")
model.save_pretrained(trained_model_save_path_final,safe_serialization=True)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, 
                                          trust_remote_code=True)
#Note: the tokenizer from the llama-2 models does not use a pad token therefore the following line is necessary:
tokenizer.pad_token = tokenizer.eos_token 

print(f"[] Saving Tokenizer at {trained_model_save_path_final}")
tokenizer.save_pretrained(trained_model_save_path_final,safe_serialization=True)

[] Saving Tokenizer at /root/LLM-models/Llama-2-13b-hf-juris/


('/root/LLM-models/Llama-2-13b-hf-juris/tokenizer_config.json',
 '/root/LLM-models/Llama-2-13b-hf-juris/special_tokens_map.json',
 '/root/LLM-models/Llama-2-13b-hf-juris/tokenizer.model',
 '/root/LLM-models/Llama-2-13b-hf-juris/added_tokens.json',
 '/root/LLM-models/Llama-2-13b-hf-juris/tokenizer.json')